# CO2e Emission WML Train Recipe¶

This notebook demonstrates the use of AI Model Factory recipes to deploy rule-based anomaly pipeline to WML


# Import Libraries & Functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# Print functions
def print_job_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    if "status" in status_data:
        print("The status of job {} is {}.".format(job_id, status_data['status']))
    else:
        print(status_data)
    if status_data['status']:
        return status_data['status']
    else:
        return "initializing"
    
def print_all_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    print(status_data)


In [ ]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))
    return summary_data

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                print('summary',summary)
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"

    # Close the HTML code
    html_code += "</div>"
    
    return html_code


# CO2 Deployment

## Defining Paths

In [ ]:
# Define the file paths
# endpoint_url = "https://tenant1.predict.masinst1.ibmmam.com/ibm/modelfactory/service"
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
        
data_file_path =  "../data/training.csv"
co2_conversion_file_path = "../data/co2e_conversion.csv"
config_file_path =  "../config/parameter_config.yaml"


## POST Response

In [ ]:
import requests

files = {
    "data_file": ("data/training.csv", open(data_file_path, 'rb')),
    "co2_conversion_file": ("data/co2e_conversion.csv", open(co2_conversion_file_path, 'rb')), 
    "config_file": ("config/parameter_config.yaml", open(config_file_path, 'rb')),
   
}

#/ibm/modelfactory/service/recipe/electrical-transformer/kpi/emission/train
#/ibm/modelfactory/service/recipe/electrical-transformer/kpi/emission
url = endpoint_url + "/recipe/electrical-transformer/kpi/emission"
post_response = requests.request("POST", url, headers={}, data={}, files=files)

In [ ]:
print(url)

In [ ]:
post_r_json = post_response.json()

anomaly_service_jobId = None

if 'jobId' in post_r_json:
    anomaly_service_jobId = post_r_json['jobId']
    print ('submitted successfully job : ', post_r_json['jobId'])
else:
    print (post_r_json)


## GET Response  - Keep Refresh it for a while until seeing the STATUS changed to DONE

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [ ]:
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']

In [ ]:
print(log_url + job_id)

In [ ]:
json_data = print_job_details(job_id,endpoint_url)

In [ ]:
json_data

In [ ]:
# Check the log
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']
get_response = requests.get(log_url+job_id, headers={})
log_data = get_response.json()
print(log_data)

### Saving details for deployment

In [ ]:
json_data["detailed_summary"]

In [ ]:
import yaml
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        for summary in json_data["detailed_summary"]:
            if summary["params.output_artifact_name"] != "":
                s3_uri = summary["artifact_uri"] + "/" + summary["params.output_artifact_name"]
            if summary['tags.artifact_name']!="":
                model_uri = "runs:/" + summary['run_id'] + "/" + summary['tags.artifact_name']
        output_data = {
            "model_uri" : model_uri,
            "train_job_id" : job_id,
            "s3_uri": s3_uri,
        }
        with open("../config/model_info.yml","w") as file:
            yaml.dump(output_data, file)
        output_data

In [ ]:
output_data